# Tensorflow Models

## mnist

In [ ]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

learning_rate = 0.01
batch_size = 32
n_epochs = 30

mnist = input_data.read_data_sets('data/mnist', one_hot=True)
X_batch, Y_batch = mnist.train.next_batch(batch_size)

print('learning_rate ', learning_rate)
print('batch_size ', batch_size)
print('n_epochs ', n_epochs)
print('X_batch', X_batch.shape, X_batch.dtype)
print('Y_batch', Y_batch.shape, X_batch.dtype)

image_size = X_batch.shape[1]
n_classes = Y_batch.shape[1]

X = tf.placeholder(dtype=tf.float32, shape=[batch_size, image_size], name='input')
Y = tf.placeholder(dtype=tf.int32, shape=[batch_size, n_classes], name='label')

n_hiden_layer = 10

w = tf.get_variable(name='weights', dtype=tf.float32, shape=[image_size, n_hiden_layer], initializer=tf.truncated_normal_initializer())
b = tf.get_variable(name='bias', dtype=tf.float32, shape=[1, n_hiden_layer], initializer=tf.truncated_normal_initializer())

logits = tf.matmul(X, w) + b
print('logits ',logits.shape)

# train
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='loss')
print('entropy ', entropy.shape)

loss = tf.reduce_mean(entropy)
print('loss ', loss.shape)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

# test
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))

writer = tf.summary.FileWriter('./graphs/logreg_placeholder', tf.get_default_graph())

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_train_batches = int(mnist.train.num_examples / batch_size)
    n_test_batches = int(mnist.test.num_examples / batch_size)

    for i in range(n_epochs):
        total_loss = 0

        # train
        for j in range(n_train_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            _, loss_batch, accuracy_batch = sess.run([optimizer, loss, accuracy], {X: X_batch, Y: Y_batch})
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}: {2}'.format(i, total_loss / n_train_batches, accuracy_batch / batch_size))

        # validation
        total_correct_preds = 0
        for j in range(n_test_batches):
            X_batch, Y_batch = mnist.test.next_batch(batch_size)
            accuracy_batch = sess.run(accuracy, {X: X_batch, Y: Y_batch})
            total_correct_preds += accuracy_batch
        print('Epoch {0} Accuracy {1}'.format(i, total_correct_preds / mnist.test.num_examples))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    # test
    total_correct_preds = 0

    for i in range(n_test_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        accuracy_batch = sess.run(accuracy, {X: X_batch, Y: Y_batch})
        total_correct_preds += accuracy_batch

    print('Total: Accuracy {0}'.format(total_correct_preds / mnist.test.num_examples))



## word2vec